In [27]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import os
import json
import folium
from folium import plugins
import re
from haversine import haversine, Unit
from datetime import datetime, date, time
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from collections import namedtuple
import geopandas as gpd
from folium.plugins import HeatMap

### Data from Bixi

In [28]:
data = pd.read_pickle("paths")

In [29]:
data

,start_station_code,end_station_code,counts,latitude_start,longitude_start,latitude_end,longitude_end,distance
0,6012,4000,65,45.510660,-73.564970,45.549457,-73.541649,4.680953
1,6012,4001,1,45.510660,-73.564970,45.502629,-73.527750,3.034886
2,6012,4002,15,45.510660,-73.564970,45.506194,-73.569968,0.631100
3,6012,5002,1,45.510660,-73.564970,45.533200,-73.515600,4.590819
4,6012,5003,2,45.510660,-73.564970,45.529400,-73.517800,4.224679
...,...,...,...,...,...,...,...,...
15463,6912,7141,3,45.535341,-73.603659,45.478889,-73.581989,6.500363
15464,6912,7147,42,45.535341,-73.603659,45.534739,-73.620769,1.334345
15465,6912,7148,2,45.535341,-73.603659,45.559884,-73.633568,3.587824
15466,6912,7149,7,45.535341,-73.603659,45.562440,-73.595333,3.082281


In [30]:
data = data.sort_values(by="counts", axis=0)

In [31]:
data[10000:]

,start_station_code,end_station_code,counts,latitude_start,longitude_start,latitude_end,longitude_end,distance
6171,6073,7031,32,45.505010,-73.570690,45.522878,-73.577495,2.056379
5148,6064,7062,32,45.500380,-73.575070,45.497024,-73.559223,1.290281
4203,6053,6096,32,45.502326,-73.565138,45.493569,-73.568576,1.009908
4205,6053,6098,32,45.502326,-73.565138,45.496420,-73.576160,1.081307
11649,6216,6237,32,45.514960,-73.585030,45.532013,-73.580541,1.928189
...,...,...,...,...,...,...,...,...
3654,6052,6026,2080,45.498030,-73.552665,45.507629,-73.551876,1.069096
7386,6136,6163,2089,45.527616,-73.589115,45.535430,-73.582200,1.022253
3938,6052,6406,2385,45.498030,-73.552665,45.480208,-73.577599,2.775789
2666,6036,6036,2528,45.504245,-73.553545,45.504245,-73.553545,0.000000


In [32]:
data.sample(150)

,start_station_code,end_station_code,counts,latitude_start,longitude_start,latitude_end,longitude_end,distance
7928,6154,6176,282,45.532290,-73.575440,45.523171,-73.573431,1.025994
383,6012,6705,2,45.510660,-73.564970,45.456365,-73.576140,6.099817
5288,6070,6106,4,45.506448,-73.576349,45.521140,-73.549260,2.669058
4051,6052,7039,1,45.498030,-73.552665,45.543763,-73.567720,5.218797
8548,6155,6311,12,45.530920,-73.576740,45.534795,-73.609546,2.591403
...,...,...,...,...,...,...,...,...
11117,6211,6234,85,45.515610,-73.575690,45.526970,-73.602610,2.448312
2536,6026,7008,1,45.507629,-73.551876,45.550769,-73.620911,7.206225
2556,6026,7032,7,45.507629,-73.551876,45.526252,-73.589758,3.605528
7049,6114,6368,14,45.523530,-73.551990,45.550316,-73.573358,3.411950


### Data from the bike paths

In [48]:
fname = "./BikePaths/reseau_cyclable_2018_c.geojson"
bp = gpd.read_file(fname)
bp = bp[bp['Ville_MTL']=='OUI']
bp = bp.drop(columns=['ID', 'ID_TRC_GEO', 'TYPE_VOIE', 'TYPE_VOIE2', 'SEPARATEUR', 'SAISONS4', 'PROTEGE_4S', 'Ville_MTL'])
bp = bp.reset_index(drop=True)

In [52]:
bp['distances'] = bp.apply(lambda x: np.array([]),axis=1)
for k in range(len(bp)):
    distances = []
    for l in range(len(bp['geometry'][k])):
        subdistances = np.array([])
        for m in range(len(bp['geometry'][k][l].xy[0])-1):
            #distances[l] = np.append(distances[l], haversine((x.latitude_start,x.longitude_start),(x.latitude_end,x.longitude_end)))
            subdistances = np.append(subdistances, 1000*haversine((bp['geometry'][k][l].xy[1][m], bp['geometry'][k][l].xy[0][m]),(bp['geometry'][k][l].xy[1][m+1], bp['geometry'][k][l].xy[0][m+1])))
        distances.append(subdistances)
    bp['distances'][k] = np.array(distances)

C:\Users\quent\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


### Useful functions

In [ ]:
def isincluded(x, y, a, b, da, db): # Attention ! da and db must be positive
    if abs(2*x - 2*a - da) <= da:
        if abs(2*y - 2*b - db) <= db:
            return 1
    return 0

In [ ]:
def calculate_truncated_distance(x1, y1, x2, y2, a, b, da, db, d0, d1): # Attention ! da and db must be positive
    # Case where the first point is in the square
    if ((d0 == 1) and (d1 ==0)):
        if x1==x2:
            if y2<b:
                xt, yt = x1, b
            else:
                xt, yt = x1, b + db
        elif y1==y2:
            if x2<a:
                xt, yt = a, y1
            else:
                xt, yt = a + da, y1
        else:
            # Case y2 < b (under the square)
            if y2 < b:
                xtest = (x2-x1)*(b-y1)/(y2-y1) + x1
                if xtest < a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                elif xtest > a + da:
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
                else : 
                    xt, yt = xtest, b
            # Case y2 > b + db (above the square)
            elif y2 > b+db:
                xtest = (x2-x1)*(b+db-y1)/(y2-y1) + x1
                if xtest < a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                elif xtest > a + da:
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
                else : 
                    xt, yt = xtest, b+db
            # Case y in the range(b, b+db)
            else:
                if x2 <a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                else: # Case x2>a+da
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
        return  1000*haversine((y1,x1),(yt,xt))
    elif ((d0 == 0) and (d1 ==1)):
        return calculate_truncated_distance(x2, y2, x1, y1, a, b, da, db, d1, d0)
    else:
        return(0.)

In [88]:
def compute_density_in_squares(xp1,yp1,xp2,yp2):
    '''(x1,y1) represents the point at the bottom left and (x2,y2) the point at the top right
    The n_x_squares number represents the number of squares between x1 and x2. 
    The length of a square according to x will then be used to calculate the number of squares according to y. '''
    
    # The point (x1, y1) must correspond to the point at the bottom left of the square.
    if xp1<=xp2:
        x1, x2 = xp1, xp2
    else:
        x1, x2 = xp2, xp1
    if yp1<=yp2:
        y1, y2 = yp1, yp2
    else:
        y1, y2 = yp2, yp1
    # We look at whether the longest length of the square is according to x or y.
    if x2-x1>y2-y1:
        y1, y2 = (y1+y2)/2 - (x2-x1)/2 , (y1+y2)/2 + (x2-x1)/2
    else:
        x1, x2 = (x1+x2)/2 - (y2-y1)/2 , (x1+x2)/2 + (y2-y1)/2
    # Convert the coordinates to a and b
    a, b = x1, y1
    da, db = x2-x1, y2-y1 # da = db
    # Calculate the total distance in this square
    total_distance_in_the_square = 0.
    for k in range(len(bp['geometry'])):
        for l in range(len(bp['geometry'][k])): # l et k sont les id d'un chemin simple
            for i in range(len(bp['geometry'][k][l].xy[0])-1): # i est l'id du point sur ce chemin
                x1, y1, x2, y2 = bp['geometry'][k][l].xy[0][i], bp['geometry'][k][l].xy[1][i] , bp['geometry'][k][l].xy[0][i+1], bp['geometry'][k][l].xy[1][i+1]
                d0, d1 = isincluded(x1, y1, a, b, da, db) , isincluded(x2, y2, a, b, da, db)
                if d0+d1==2: # Si les deux points sont dans le carré, on ajoute le premier
                    total_distance_in_the_square += bp['distances'][k][l][i]
                elif ((d1==1) or (d0==1)): # Si seul le deuxième point est dans le carré, on ajoute l'intersection seulement
                    total_distance_in_the_square += calculate_truncated_distance(x1, y1, x2, y2, a, b, da, db, d0, d1)
    total_distance_in_the_square = (total_distance_in_the_square*1e-3)/((haversine((b, a), (b+db, a)))**2) # unit : km / (km**2)
    return total_distance_in_the_square

In [89]:
subdata = data.sample(3)

In [90]:
subdata['bp_density'] = bp.apply(lambda x: 0.,axis=1)

In [91]:
subdata

,start_station_code,end_station_code,counts,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
14306,6411,6415,3,45.513338,-73.572950,45.470300,-73.616200,5.853864,NaN
8273,6154,7141,6,45.532290,-73.575440,45.478889,-73.581989,5.959876,NaN
3726,6052,6109,37,45.498030,-73.552665,45.513413,-73.538253,2.046219,0.0


In [92]:
import time
start_time = time.time()
for k in subdata.index:
    subdata['bp_density'][k] = compute_density_in_squares(subdata['longitude_start'][k],subdata['latitude_start'][k],subdata['longitude_end'][k],subdata['latitude_end'][k])
elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

C:\Users\quent\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


'00:00:24'

In [93]:
subdata

,start_station_code,end_station_code,counts,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
14306,6411,6415,3,45.513338,-73.572950,45.470300,-73.616200,5.853864,1.159069
8273,6154,7141,6,45.532290,-73.575440,45.478889,-73.581989,5.959876,2.210323
3726,6052,6109,37,45.498030,-73.552665,45.513413,-73.538253,2.046219,1.104594
